In [1]:
# Connexion à l'environnement 

from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

ml_client = MLClient.from_config(credential=DefaultAzureCredential())

Found the config file in: /config.json


In [ ]:
# TARGET OF THIS NOTEBOOK
# PART 1 : HUGGING FACE MODEL FROM MODEL CATALOG (azure ML)
#==> deepset-gbert-base


# PART 2 : HUGGING FACE MODEL FROM WEBSITE
#==> https://huggingface.co/Qwen/Qwen2-VL-7B-Instruct  |  



In [2]:
# PART 1 Hugging Face from Model Catalog

In [ ]:
# Config
from azure.ai.ml import MLClient
from azure.ai.ml import MLClient, Input
from azure.ai.ml.entities import (
ManagedOnlineEndpoint,
ManagedOnlineDeployment,
Model,
Environment,
CodeConfiguration,
)
from azure.identity import DefaultAzureCredential
from azure.ai.ml.constants import AssetTypes


registry_name = "HuggingFace"
model_name = "deepset-gbert-base"
version = "5"
model_local_path = f"azureml://registries/{registry_name}/models/{model_name}/versions/{version}"
print(model_local_path)



azureml://registries/HuggingFace/models/deepset-gbert-base/versions/5


In [ ]:
# création d'un endpoint

import datetime

# Nom unique pour l'endpoint (on peut suffixer avec la date/heure)
endpoint_name = "my-endpoint-" + datetime.datetime.now().strftime("%Y%m%d%H%M")

endpoint = ManagedOnlineEndpoint(
    name=endpoint_name,
    description="Endpoint pour démo - déploie un modèle depuis le registry",
    auth_mode="key"  # ou "aml_token"
)

# Crée l'endpoint (ou le met à jour s'il existe déjà)
endpoint = ml_client.begin_create_or_update(endpoint).result()
print(f"Endpoint créé : {endpoint.name}")

Endpoint créé : my-endpoint-202501231312


In [23]:
# Déploiement sur l'endpoint

deployment_name = "demo-deployment"

deployment = ManagedOnlineDeployment(
    name=deployment_name,
    endpoint_name=endpoint_name,
    model=model_local_path,
    # Vous pouvez référencer un environnement existant dans votre workspace
    environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu:1",  
    instance_type="Standard_DS3_v2",
    instance_count=1
)

deployment = ml_client.begin_create_or_update(deployment).result()
print(f"Déploiement créé : {deployment.name}")

# (Optionnel) Basculez tout le trafic vers ce déploiement
endpoint.traffic = {deployment_name: 100}
endpoint = ml_client.begin_create_or_update(endpoint).result()

Check: endpoint my-endpoint-202501231312 exists


..............................................................................................................Déploiement créé : demo-deployment


Readonly attribute principal_id will be ignored in class <class 'azure.ai.ml._restclient.v2022_05_01.models._models_py3.ManagedServiceIdentity'>
Readonly attribute tenant_id will be ignored in class <class 'azure.ai.ml._restclient.v2022_05_01.models._models_py3.ManagedServiceIdentity'>


In [ ]:
# more examples, testing here : https://github.com/azure/azureml-examples/blob/main/sdk/python/foundation-models/huggingface/inference/


In [ ]:
# PART 2 : HUGGING FACE MODEL FROM WEBSITE
#==> https://huggingface.co/Qwen/Qwen2-VL-7B-Instruct  | 

In [2]:
from huggingface_hub import snapshot_download

LOCAL_MODEL_DIR = "./Qwen2-VL"

snapshot_download(
    repo_id="Qwen/Qwen2-VL-7B-Instruct",
    local_dir=LOCAL_MODEL_DIR,
)

Fetching 17 files:   0%|          | 0/17 [00:00<?, ?it/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/3.90G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/244 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/347 [00:00<?, ?B/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/1.09G [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

'/mnt/batch/tasks/shared/LS_root/mounts/clusters/jcharley2/code/Users/Saint-gobain/Qwen2-VL'

In [3]:
# registration in azure ml

from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
from azure.ai.ml.entities import Model

ml_client = MLClient.from_config(credential=DefaultAzureCredential())

model_entity = Model(
    name="Qwen2-VL",
    path=LOCAL_MODEL_DIR,  # local folder with all the downloaded files
    description="A Hugging Face model downloaded locally and registered in Azure ML"
)

registered_model = ml_client.models.create_or_update(model_entity)
print(f"Model registered successfully!")
print(f"Name: {registered_model.name}, Version: {registered_model.version}")
print(f"Model ID: {registered_model.id}")

Found the config file in: /config.json


Overriding of current TracerProvider is not allowed
Overriding of current LoggerProvider is not allowed
Overriding of current MeterProvider is not allowed
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Your file exceeds 100 MB. If you experience low speeds, latency, or broken connections, we recommend using the AzCopyv10 tool for this file transfer.

Example: azcopy copy '/mnt/batch/tasks/shared/LS_root/mounts/clusters/jcharley2/code/Users/Saint-gobain/Qwen2-VL' 'https://stfqm45izfylnxqprod.blob.core.windows.net/azureml-blobstore-24beeade-8ebc-43d6-ba31-3bdbe9a007dc/LocalUpload/7cf4eab683d9d4ae74a14bd2fe5d7221/Qwen2-VL' 

See https://docs.microsoft.com/azure/storage/common/storage-use-azcopy-v10 for more information.
Uploadin

Model registered successfully!
Name: Qwen2-VL, Version: 1
Model ID: /subscriptions/bf701399-a63b-44aa-bbbf-b848add18b4d/resourceGroups/rg-mlopsdemo-0001prod/providers/Microsoft.MachineLearningServices/workspaces/mlw-mlopsdemo-0001prod/models/Qwen2-VL/versions/1


In [1]:
%%writefile score.py
import os
import json
import torch
from transformers import AutoTokenizer, AutoModelForVision2Seq
from accelerate import infer_auto_device_map

def init():
    """
    Initialize the model and tokenizer while avoiding memory overload.
    """
    global model, tokenizer

    # Get model path
    base_model_path = os.getenv("AZUREML_MODEL_DIR", "/var/azureml-app/azureml-models/Qwen2-VL/1")
    model_path = os.path.join(base_model_path, "Qwen2-VL")

    print(f"🔍 Checking model directory: {model_path}")

    if not os.path.exists(model_path):
        raise RuntimeError(f"Model path {model_path} not found!")

    config_path = os.path.join(model_path, "config.json")
    if not os.path.exists(config_path):
        raise RuntimeError(f"config.json not found in {model_path}!")

    print(f"config.json found at {config_path}")

    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

    # Use `device_map="auto"` to manage memory safely
    model = AutoModelForVision2Seq.from_pretrained(
        model_path,
        device_map="auto",  # Ensures it loads on available memory
        torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
        offload_folder="/tmp/model_offload",  # Offload large weights to disk
        offload_state_dict=True,  # Helps avoid memory fragmentation
        trust_remote_code=True
    )

    print("Model and tokenizer loaded successfully!")

def run(raw_data):
    """
    Process an inference request.
    """
    try:
        data = json.loads(raw_data)
        prompt = data.get("prompt", "Hello, how can I assist you?")

        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
        outputs = model.generate(**inputs, max_new_tokens=128)

        response = tokenizer.decode(outputs[0], skip_special_tokens=True)

        return {"response": response}

    except Exception as e:
        return {"error": str(e)}



Overwriting score.py


In [10]:
%%writefile environment.yml
name: qwen2vl_env
channels:
  - defaults
  - conda-forge
dependencies:
  - python=3.9
  - pip
  - pip:
    - torch>=2.0
    - transformers>=4.26
    - accelerate>=0.16
    - safetensors
    - sentencepiece
    - azureml-inference-server-http


Overwriting environment.yml


In [2]:
import datetime
import requests
import json

from azure.ai.ml import MLClient
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Environment,
    CodeConfiguration
)
from azure.identity import DefaultAzureCredential

# 1) Connexion au workspace 
ml_client = MLClient.from_config(credential=DefaultAzureCredential())

# 2) Création d'un endpoint  unique
endpoint_name = "qwen2vl-endpoint-" + datetime.datetime.now().strftime("%Y%m%d%H%M")
endpoint = ManagedOnlineEndpoint(
    name=endpoint_name,
    description="Endpoint pour Qwen2-VL-7B-Instruct",
    auth_mode="key"
)

endpoint = ml_client.begin_create_or_update(endpoint).result()
print(f"Endpoint : {endpoint.name}")

# 3) Création d'un environnement
env = Environment(
    name="qwen2vl-env",
    description="Env pour Qwen2-VL-7B-Instruct",
    conda_file="environment.yml",
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest"
)
registered_env = ml_client.environments.create_or_update(env)
print("Environnement enregistré :", registered_env.name)

# 4) Code + scoring config
code_config = CodeConfiguration(
    code=".",            
    scoring_script="score.py"
)

# 5) Déploiement (ManagedOnlineDeployment) depuis le modèle enregistré
model_name = "Qwen2-VL" 
model_version = "1"  #  "latest" 
model_path_in_aml = f"azureml:{model_name}:{model_version}"

deployment_name = "qwen2vl"
deployment = ManagedOnlineDeployment(
    name=deployment_name,
    endpoint_name=endpoint_name,
    environment=env,
    code_configuration=code_config,
    instance_type="Standard_NC6s_v3",
    instance_count=1,
    # Ici, on passe le modèle enregistré
    model=model_path_in_aml  
)

deployment = ml_client.begin_create_or_update(deployment).result()
print(f"Déploiement : {deployment.name}")

# 6) Diriger le trafic à 100% vers le déploiement
endpoint.traffic = {deployment_name: 100}
endpoint = ml_client.begin_create_or_update(endpoint).result()
print(f"Trafic routé à 100% vers : {deployment.name}")

# 7) Test d'inférence
endpoint_key = ml_client.online_endpoints.get_keys(endpoint_name).primary_key
endpoint_uri = endpoint.scoring_uri

headers = {
    "Authorization": f"Bearer {endpoint_key}",
    "Content-Type": "application/json"
}

payload = {
    "prompt": "Présente-toi en français."
}

resp = requests.post(endpoint_uri, headers=headers, data=json.dumps(payload))
print("\n=== Réponse du modèle ===")
print(resp.text)


Found the config file in: /config.json


Endpoint : qwen2vl-endpoint-202501292031


Check: endpoint qwen2vl-endpoint-202501292031 exists


Environnement enregistré : qwen2vl-env


Uploading code (0.03 MBs): 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 27757/27757 [00:00<00:00, 393643.67it/s]


.....................................................................................................................................

Readonly attribute principal_id will be ignored in class <class 'azure.ai.ml._restclient.v2022_05_01.models._models_py3.ManagedServiceIdentity'>
Readonly attribute tenant_id will be ignored in class <class 'azure.ai.ml._restclient.v2022_05_01.models._models_py3.ManagedServiceIdentity'>


Déploiement : qwen2vl
Trafic routé à 100% vers : qwen2vl

=== Réponse du modèle ===



In [38]:
#Option alternative : Déploiement en VLLM